In [19]:
from astroquery.ipac.ned import Ned
import numpy as np
import os
import urllib.request
from astroquery.gaia import Gaia
from astropy import log
import pandas as pd
import os
import glob
import numpy as np
import importlib
from matplotlib import colors
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

def fit_line(x, y):
    # given one dimensional x and y vectors - return x and y for fitting a line on top of the regression
    # inspired by the numpy manual - https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html 
    x = x.to_numpy() # convert into numpy arrays
    y = y.to_numpy() # convert into numpy arrays

    A = np.vstack([x, np.ones(len(x))]).T # sent the design matrix using the intercepts
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]

    return m, c

hubble_objs = { "S. Mag", "L. Mag",
 "NGC 278", "NGC 404", "NGC 584", "NGC 936", "NGC 1023", "NGC 1700", "NGC 2681", "NGC 2683", "NGC 2841", 
 "NGC 3034", "NGC 3115", "NGC 3368", "NGC 3379", "NGC 3489", "NGC 3521", "NGC 3623", "NGC 4111", "NGC 4526",
 "NGC 4565", "NGC 4594", "NGC 5005", "NGC 5866",
 "NGC 6822", "NGC 598", "NGC 221", "NGC 224", 
    "NGC 5457", "NGC 4736", "NGC 5194", "NGC 4449", "NGC 4214", "NGC 3031", "NGC 3627",
    "NGC 4826", "NGC 5236", "NGC 1068", "NGC 5055", "NGC 7331", "NGC 4258", "NGC 4151",
    "NGC 4382", "NGC 4472", "NGC 4486", "NGC 4649" }

print("Rows: {}".format(len(hubble_objs)))
distances = pd.read_csv("NED-D.csv")

def get_obj(m):
    try:
        obj_data = Ned.query_object(m).to_pandas().iloc[0]

        d = distances.loc[distances['ID'] == obj_data['Object Name']]
        if d.empty:
            return None
        mpc = d.sort_values(by=['Error']).iloc[0]['MPC']

        return pd.DataFrame([[obj_data['Object Name'], obj_data['Velocity'], mpc]], columns=['Name', 'Velocity', 'MPC'])
    except:
        return None

i = 0
step = 50
while i < 9999:
    file_name = "data/obj_{}_{}.csv".format(i, i+step)
    if os.path.isfile(file_name):
        log.info("Skipping: " + file_name)
    else:
        log.info("Downloading data for: " + file_name)
        objects = []
        for y in range(i, i+step):
            objects.append("NGC {}".format(y))

        try:
            data = pd.concat(mo for obj in objects if (mo:=get_obj(obj)) is not None)
            data.to_csv(file_name)
        except:
            {}

    i = i + step


# Grab the previously calculated results
data = pd.concat(pd.read_csv(f) for f in glob.glob("data/obj_*.csv"))
data.dropna()
data = data[data['MPC'] > 0]
#data = data[data['MPC'] < 100]
data = data[data['Velocity'] > 0]
#data = pd.concat(mo for obj in hubble_objs if (mo:=get_obj(obj)) is not None)

data.index = pd.RangeIndex(0, data.shape[0])

data.Velocity = data.Velocity.abs()
display(data)

# plot the data as a scatter plot
fig = px.scatter(x=data['MPC'], y=data['Velocity'], opacity=.5, labels={"x" : "Distance (MPC)", "y" : "Velocity (Km/s)"}) 

# fit a linear model 
m, c = fit_line(x = data['MPC'], y = data['Velocity'])

# add the linear fit on top
fig.add_trace(
    go.Scatter(
        x=data['MPC'],
        y=m*data['MPC'] + c,
        mode="lines",
        line=go.scatter.Line(color="red"),
        showlegend=False)
)
# optionally you can show the slop and the intercept 
mid_point = data['MPC'].mean()
fig.update_layout(
    showlegend=False,
    annotations=[
        go.layout.Annotation(
            x=mid_point,
            y=m*mid_point + c,
            xref="x",
            yref="y",
            text=str(round(m, 4))+'x+'+str(round(c, 2)) ,
        )
    ]
)
fig.show()
#fig.write_image('hubble_constant.png')

Rows: 46
INFO: Skipping: data/obj_0_50.csv [unknown]
INFO: Skipping: data/obj_50_100.csv [unknown]
INFO: Skipping: data/obj_100_150.csv [unknown]
INFO: Skipping: data/obj_150_200.csv [unknown]
INFO: Skipping: data/obj_200_250.csv [unknown]
INFO: Skipping: data/obj_250_300.csv [unknown]
INFO: Skipping: data/obj_300_350.csv [unknown]
INFO: Skipping: data/obj_350_400.csv [unknown]
INFO: Skipping: data/obj_400_450.csv [unknown]
INFO: Skipping: data/obj_450_500.csv [unknown]
INFO: Skipping: data/obj_500_550.csv [unknown]
INFO: Skipping: data/obj_550_600.csv [unknown]
INFO: Skipping: data/obj_600_650.csv [unknown]
INFO: Skipping: data/obj_650_700.csv [unknown]
INFO: Skipping: data/obj_700_750.csv [unknown]
INFO: Skipping: data/obj_750_800.csv [unknown]
INFO: Skipping: data/obj_800_850.csv [unknown]
INFO: Skipping: data/obj_850_900.csv [unknown]
INFO: Skipping: data/obj_900_950.csv [unknown]
INFO: Skipping: data/obj_950_1000.csv [unknown]
INFO: Skipping: data/obj_1000_1050.csv [unknown]
INFO:

,Unnamed: 0,Name,Velocity,MPC
0,0,NGC 0001,4550.0,74.1
1,0,NGC 0002,7547.0,88.3
2,0,NGC 0007,1495.0,19.5
3,0,NGC 0009,4528.0,35.4
4,0,NGC 0010,6804.0,106.0
...,...,...,...,...
3755,0,NGC 0991,1532.0,18.8
3756,0,NGC 0992,4132.0,60.3
3757,0,NGC 0993,6912.0,95.3
3758,0,NGC 0993,6912.0,95.3
